In [1]:
from collections import defaultdict, deque
import numpy as np
import pandas as pd
from math import factorial

nodes_list = np.array([[ 1,  2], [ 2,  3], [ 3,  4], [ 4,  5], [ 5,  1],
                       [ 6,  7], [ 7,  8], [ 8,  9], [ 9, 10], [10, 11],
                       [11, 12], [12, 13], [13, 14], [14, 15], [15,  6],
                       [16, 17], [17, 18], [18, 19], [19, 20], [20, 16],
                       [15, 16], [ 7, 17], [ 9, 18], [11, 19], [13, 20],
                       [ 1,  8], [ 2, 10], [ 3, 12], [ 4, 14], [ 5,  6]])
dir_path = r'C:\Users\RedmiBook\Documents\SCE\third_1\Ann\\'

In [2]:
def build_start_graph():    
    graph = defaultdict(set)
    for edge in nodes_list:
        a, b = edge[0], edge[1]
        graph[a].add(b)
        graph[b].add(a)
    return graph

In [3]:
def check_way(res_graph, a, b):
    que_for_check = deque(res_graph[a])
    res_set = {a}
    checked_node = {a}
    while que_for_check and b not in res_set:
        elem = que_for_check.popleft() 
        if elem not in checked_node: 
            res_set = res_set | res_graph[elem]
            que_for_check.extend(list(res_graph[elem]))
            checked_node = checked_node | {elem}
    return b in res_set

In [4]:
def calc_f(M, a, b, c):
    res_arr = np.zeros(30)
    for _ in range(M):
        res_s = np.random.permutation(nodes_list)
        st_g = build_start_graph()    
        for i_r, rib in enumerate(res_s):
            st_g[rib[0]].remove(rib[1])
            st_g[rib[1]].remove(rib[0])
            if not check_way(st_g, a, b) or not check_way(st_g, c, b):
                finish = i_r
                break
        res_arr[finish] += 1
    return res_arr / M

In [5]:
def calc_F(p, n=30):
    f_n = factorial(n)
    q = 1 - p
    F = []
    for k in range(1, n + 1):
        res_sum = 0
        for i in range(k, n + 1):
            res_sum += (f_n / (factorial(i) * factorial(n - i))) * ((q) ** i) * ((p) ** (n - i))
        F.append(res_sum)
    return F

In [304]:
N = 30
a, b, c = 1, 13, 17
n_list = [1000, 10000, 50000]
p_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
F = np.zeros((len(p_list), N))
f = np.zeros((len(n_list), N))
for i_p, p in enumerate(p_list):
    F[i_p] = calc_F(p)
for i_n, n in enumerate(n_list):
    f[i_n] = calc_f(n, a, b, c)
# distr_spectr = pd.DataFrame(f.T, index=range(1, N + 1), columns=n_list)
# distr_spectr.to_csv(dir_path + 'distr_spectr.csv')
# distr_spectr

In [165]:
def cum_spec(f, N=30):
    cum_s = np.zeros((len(n_list)-1, N))
    for i_m, f_m in enumerate(f[:-1]):
        for i in range(1, N):
            cum_s[i_m][i] = cum_s[i_m][i - 1] + f_m[i_m][i]
    return cum_s

In [25]:
pd.DataFrame(cum_spec(f).T, index=range(1, N + 1), columns=n_list[:-1])

,1000,10000
1,0.000,0.0000
2,0.000,0.0000
3,0.000,0.0009
4,0.003,0.0025
5,0.008,0.0081
6,0.020,0.0197
7,0.045,0.0380
8,0.088,0.0781
9,0.158,0.1432
10,0.259,0.2470


In [168]:
def calc_Y_Z(M, a, b, c, N=30):
    a_list = np.zeros(N)
    b_list = np.zeros((N, N))
    for _ in range(M):
        random_permutation = np.random.permutation(range(N))
        st_g = build_start_graph()    
        for i_r, rib in enumerate(random_permutation):
            st_g[nodes_list[rib][0]].remove(nodes_list[rib][1])
            st_g[nodes_list[rib][1]].remove(nodes_list[rib][0])
            if not check_way(st_g, a, b) or not check_way(st_g, c, b):
                finish = i_r
                for i in range(i_r, N):
                    b_list[i][random_permutation[i]] += 1
                break
        a_list[finish] += 1
    return a_list / M, b_list / M

In [169]:
Y_1000, Z_1000 = calc_Y_Z(1000, a, b, c)
Y_10000, Z_10000 = calc_Y_Z(10000, a, b, c)

In [174]:
def cum_sum(ser):
    res_ser = np.zeros(len(ser))
    for i in range(1, len(res_ser)):
        res_ser[i] = res_ser[i - 1] + ser[i]
    return res_ser

In [181]:
less_imp = [18 - 1, 24 - 1]
most_imp = [13 - 1, 26 - 1]
pd.DataFrame([cum_sum(Z_1000[less_imp[0]]), cum_sum(Z_10000[less_imp[0]]), 
              cum_sum(Z_1000[less_imp[1]]), cum_sum(Z_10000[less_imp[1]]),
              cum_sum(Z_1000[most_imp[0]]), cum_sum(Z_10000[most_imp[0]]), 
              cum_sum(Z_1000[most_imp[1]]), cum_sum(Z_10000[most_imp[1]])], 
             index=['18 M=1000', '18 M=10000', '24 M=1000', '24 M=10000',
                     '13 M=1000', '13 M=10000', '26 M=1000', '26 M=10000'], columns=range(1, 31)).T

,18 M=1000,18 M=10000,24 M=1000,24 M=10000,13 M=1000,13 M=10000,26 M=1000,26 M=10000
1,0.000,0.0000,0.000,0.0000,0.000,0.0000,0.000,0.0000
2,0.032,0.0307,0.037,0.0322,0.017,0.0240,0.028,0.0349
3,0.061,0.0672,0.068,0.0636,0.035,0.0477,0.055,0.0691
4,0.099,0.0948,0.097,0.0999,0.046,0.0707,0.086,0.1041
5,0.132,0.1279,0.127,0.1346,0.081,0.0948,0.122,0.1391
6,0.160,0.1596,0.160,0.1695,0.097,0.1198,0.154,0.1716
7,0.192,0.1936,0.194,0.2024,0.128,0.1412,0.184,0.2020
8,0.217,0.2240,0.221,0.2352,0.149,0.1626,0.223,0.2374
9,0.262,0.2564,0.260,0.2652,0.177,0.1857,0.247,0.2659
10,0.296,0.2896,0.300,0.3007,0.196,0.2075,0.283,0.2982


In [238]:
def calc_BIM(Z, Y, p, n=30):
    bim = np.zeros(np.shape(Z)[1])    
    q = 1 - p
    mul = factorial(n)
    for j in range(len(bim)):
        sum_bim = 0.0
        for i in range(1, n + 1):
            numerator = (Z[i - 1, j - 1] * (p ** (i - 1)) * (q ** (n - i))) - ((Y[i - 1] - Z[i - 1, j - 1]) * (p ** i) * (q ** (n - i - 1))) 
            sum_bim += numerator * mul / (factorial(i) * factorial(n - i))
        bim[j] = abs(sum_bim)
    return bim

In [267]:
n = 30
bim_arr = np.zeros((6, n))
bim_arr_p = np.zeros((6, n))
for i, p in enumerate([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    bim_arr[i] = (calc_BIM(Z_1000, Y_1000, p))
    bim_arr_p[i] = (bim_arr[i] * (1 - p))
# bim_arr

In [269]:
pd.DataFrame(np.array([bim_arr[:, less_imp[0]], bim_arr_p[:, less_imp[0]], 
              bim_arr[:, less_imp[1]], bim_arr_p[:, less_imp[1]],
              bim_arr[:, most_imp[0]], bim_arr_p[:, most_imp[0]], 
              bim_arr[:, most_imp[1]], bim_arr_p[:, most_imp[1]]]).T, 
             columns=['BIM a', 'BIM a * p', 'BIM b', 'BIM b * p', 
                      'BIM c', 'BIM c * p', 'BIM d', 'BIM d * p'],
             index=['0.4', '0.5', '0.6', '0.7', '0.8', '0.9'])

,BIM a,BIM a * p,BIM b,BIM b * p,BIM c,BIM c * p,BIM d,BIM d * p
0.4,0.096633,0.057980,0.110303,0.066182,0.096945,0.058167,0.089797,0.053878
0.5,0.049711,0.024855,0.055860,0.027930,0.047137,0.023568,0.049404,0.024702
0.6,0.047126,0.018850,0.053373,0.021349,0.046852,0.018741,0.047307,0.018923
0.7,0.134533,0.040360,0.141097,0.042329,0.120781,0.036234,0.129344,0.038803
0.8,0.220384,0.044077,0.207213,0.041443,0.200247,0.040049,0.200027,0.040005
0.9,0.361249,0.036125,0.375609,0.037561,0.366699,0.036670,0.372307,0.037231


In [290]:
def build_graph_monte(p, rib_n, rib_inc=True):    
    edges = (nodes_list.T * (np.random.rand(30) < [p]).astype(int)).T
    graph = defaultdict(set)
    for edge in edges:
        a, b = edge[0], edge[1]
        graph[a].add(b)
        graph[b].add(a)
    if rib_inc:
        graph[nodes_list[rib_n][0]].add(nodes_list[rib_n][1])
        graph[nodes_list[rib_n][1]].add(nodes_list[rib_n][0])
    else:
        graph[nodes_list[rib_n][0]].remove(nodes_list[rib_n][1])
        graph[nodes_list[rib_n][1]].remove(nodes_list[rib_n][0])
    return graph

In [294]:
def count_res_monte(a, b, c, p, rib_n, N=1000, rib_inc=True):
    positive = 0
    for _ in range(N):
        res_graph = build_graph_monte(p, rib_n, rib_inc)
        if  check_way(res_graph, a, b) and check_way(res_graph, a, c):
            positive += 1
    return positive / N

In [302]:
res_arr_monte = np.zeros([4, 6])

for i_n, rib_n in enumerate([less_imp[0],  less_imp[1], most_imp[0], most_imp[1]]):
    for i_p, p in enumerate([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
        res_arr_monte[i_n][i_p] = abs(count_res_monte(a, b, c, p, rib_n, N=10000, rib_inc=True) - count_res_monte(a, b, c, p, rib_n, N=10000, rib_inc=True))

In [303]:
df_monte = pd.DataFrame(res_arr_monte.T, columns=['node a', 'node b', 'node c', 'node d'], 
                index=['0.4', '0.5', '0.6', '0.7', '0.8', '0.9'])
df_monte

,node a,node b,node c,node d
0.4,0.0014,0.0003,0.0024,0.0031
0.5,0.0032,0.0062,0.0038,0.0111
0.6,0.0001,0.0099,0.0041,0.0028
0.7,0.0039,0.0080,0.0023,0.0046
0.8,0.0027,0.0034,0.0042,0.0002
0.9,0.0009,0.0011,0.0004,0.0013
